In [ ]:
# 1. 라이브러리 및 터널링 프로그램 설치
!pip install streamlit scikit-learn -q
!npm install -g localtunnel

# 2. 접속 Password 확인 (이 숫자를 복사하세요!)
print("👇 아래 숫자를 복사해두세요 (Password) 👇")
!wget -q -O - ipv4.icanhazip.com

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# 페이지 설정
st.set_page_config(page_title="AI 성적 예측 시뮬레이터", layout="wide")
st.title("🎓 [Colab] AI 성적 예측 & 학습 분석기")
st.markdown("---")

# 1. 사이드바: 데이터 업로드
st.sidebar.header("1. 설정 (Settings)")
uploaded_file = st.sidebar.file_uploader("학습 로그 파일 업로드 (csv/xlsx)", type=['csv', 'xlsx'])

if uploaded_file:
    try:
        df = pd.read_csv(uploaded_file) if uploaded_file.name.endswith('csv') else pd.read_excel(uploaded_file)
    except:
        st.error("파일을 읽을 수 없습니다.")
else:
    data = {
        'Study_Hours': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'Score': [45, 50, 65, 70, 75, 85, 88, 92, 96, 100]
    }
    df = pd.DataFrame(data)
    st.sidebar.info("💡 기본 예시 데이터를 사용합니다.")

# 2. 머신러닝 학습
X = df[['Study_Hours']] # 이름표('Study_Hours')가 붙은 데이터
y = df['Score']

model = LinearRegression()
model.fit(X, y)
coef = model.coef_[0]

# 3. 사용자 시뮬레이션
st.sidebar.header("2. 예측 시뮬레이션")
input_hours = st.sidebar.slider("오늘 공부할 시간은?", 0.0, 12.0, 3.5)

#a 예측할 때도 이름표(DataFrame)를 붙여서 전달
input_df = pd.DataFrame({'Study_Hours': [input_hours]})
predicted_score = model.predict(input_df)[0]

# 점수 범위 제한 (0~100점)
predicted_score = min(max(predicted_score, 0), 100)

# 4. 메인 화면 구성
col1, col2 = st.columns([2, 1])

with col1:
    st.subheader("📊 학습 시간 vs 성적 회귀 분석")
    fig, ax = plt.subplots(figsize=(10, 6))

    # 실제 데이터
    ax.scatter(df['Study_Hours'], df['Score'], color='blue', label='Actual Data', s=100, alpha=0.6)

    # [수정 2] 그래프 선 그릴 때도 이름표(DataFrame) 붙이기!
    line_x = pd.DataFrame({'Study_Hours': [0, 12]}) # 0시간부터 12시간까지
    line_y = model.predict(line_x)

    ax.plot(line_x['Study_Hours'], line_y, color='red', linestyle='--', label='AI Regression Line')

    # 예측값 표시
    ax.scatter([input_hours], [predicted_score], color='green', s=300, marker='*', zorder=5, label='Your Prediction')

    ax.set_xlabel("Study Hours")
    ax.set_ylabel("Score")
    ax.legend()
    ax.grid(True, alpha=0.3)

    st.pyplot(fig)

with col2:
    st.subheader("🤖 AI 분석 리포트")
    st.metric(label="예상 점수", value=f"{predicted_score:.1f}점")

    st.info(f"""
    **AI 분석 결과:**
    학습 시간을 **1시간** 늘릴 때마다,
    점수는 약 **{coef:.1f}점** 오를 것으로 예측됩니다.
    """)

    if predicted_score >= 90:
        st.success("탁월합니다! A등급 예상 🏆")
    elif predicted_score >= 70:
        st.warning("좋아요! 고득점 가능 🔥")
    else:
        st.error("학습량이 더 필요해 보입니다 📚")

st.subheader("📋 학습 데이터 원본")
st.dataframe(df)

Overwriting app.py


In [ ]:
print("🚀 잠시 후 'your url is: ...' 옆의 링크가 뜨면 클릭하세요!")
!streamlit run app.py & sleep 3 && lt --port 8501